# Influencia del ingreso económico en el rendimiento en enseñanza media.


Durante los últimos años se ha notado un problema en el desempeño de los estudiantes, especialmente en contexto de pandemia, que ha traido consigo muchos aspectos negativos y han impactado significativamente las vidas de la población. Un estudio de Mineduc y el Banco de Chile publicado en agosto del año 2020* señala que debido al estado del país durante este período es probable un retroceso de 1,3 años en la escolaridad según aprendizaje de los estudiantes, lo que significa una pérdida importante tanto para establecimientos escolares como para universidades. Estando experimentando esta problemática directamente, fue considerado relevante estudiar las posibles variables que estarían influyendo en el aspecto académico del país, haciendo un énfasis en los bienes por región.

Para lo anterior, a partir de la base de datos de Mineduc y su información respecto al desempeño estudiantil, se analizarán y compararán las relaciones que esto tiene con los ingresos económicos regionales y la cantidad de redes a internet fijas, debido a la relevancia que ambas cosas han adquirido en nuestro contexto. De esa forma, se dará respuesta a las siguientes preguntas:

* ¿Cuáles son las regiones que presentan mayor desempeño académico y cuáles son las diferencias con las que tienen peor rendimiento? ¿Coincide lo anterior con el ingreso que estas presentan?
* ¿Se puede decir que hay una relación lineal entre ambas variables?
* ¿Se puede afirmar que el acceso a internet y los ingresos económicos influyen en los estudiantes?
* ¿Existe alguna conexión además entre el acceso a internet y los ingresos económicos?
* ¿Se puede predecir el desempeño del próximo año a partir de lo observado? Y si es así, ¿cómo sería?


*https://www.mineduc.cl/impacto-del-covid-19-en-los-resultados-de-aprendizaje-en-chile/

Los datos a utilizar provienen de bases públicas, y tienen distintas características:

El primero es un archivo .xlsx que se encuentra en la página web de Mineduc, y contiene información de todo tipo del desempeño estudiantil, incluyendo el promedio general anual, la comuna y región de cada estudiante. La mayoría de sus datos son de tipo numérico, incluso los que no tienen un valor específico y en su lugar lo utilizan como representación.

El segundo es un archivo de formato .csv de la Subsecrretaría de Comunicaciones que tiene información respecto a las redes de internet fijas en los hogares, clasificadas por año y región. Los elementos son de tipo numérico y es de corta extensión.

Por último, el tercer grupo de datos corresponde a un archivo .xlsx de la encuesta Casen que contiene información sobre el ingreso promedio de los hogares por región, con 7 años registrados a partir del 2006. Al ser ingresos monetarios, al igual que los datos anteriores, estos también son de tipo numérico.

## Importación de librerías:

In [30]:
import pandas as pd
import numpy as np

## Limpieza de datos

### Ingresos de los hogares por región

In [2]:
excel = pd.read_excel('Propuesta/Ingreso_Casen_en_Pandemia_2020.xlsx', '4')
excel[0:10]

,Indice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ingreso promedio de los hogares por tipo de in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Ingreso promedio de los hogares por tipo de in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2006-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2006-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,($ noviembre de cada año),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,($ noviembre de 2020),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,2006.0000,2009.0000,2011.000000,2013.0000,2015.000000,2017.000000,2020.000000,...,NaN,NaN,NaN,2006.000000,2009.000000,2011.000000,2013.000000,2015.000000,2017.000000,2020.000000
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Ingreso del trabajo2,I,Estimación,50156.3830,40113.7150,56169.042125,63696.5440,73046.327352,71980.380233,7023.837107,...,Ingreso del trabajo2,I,Estimación,79949.274502,55918.518710,73469.107099,79684.376544,83199.766854,78170.692933,7023.837107
8,NaN,NaN,Error estándar,1007.8439,1116.1195,2054.145000,1473.9436,1430.449000,1683.905940,348.498057,...,NaN,NaN,Error estándar,1606.503177,1555.870583,2686.821660,1843.903444,1629.281411,1828.721851,348.498057
9,NaN,II,Estimación,128154.1200,143229.2700,159783.860368,191072.9000,224927.592322,235981.996295,114883.663633,...,NaN,II,Estimación,204277.667280,199661.602380,208997.289362,239032.197900,256192.527655,256276.447977,114883.663633


In [3]:
names = []
for a in excel.keys():
    names.append(str(excel[a][5])[0:4])

dicti = {}
cont = 0
for a in excel.keys():
    if cont == 0 or cont == 1:
        dicti[a] = ''
    else:
        dicti[a] = names[cont]
    cont += 1
excel = excel.rename(dicti, axis = 'columns')

In [4]:
excel.columns.duplicated()

array([False,  True, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [5]:
excel_3 = excel
excel_3 = excel_3.iloc[:,~excel_3.columns.duplicated(keep = 'last')]

In [6]:
excel_3 = excel_3.drop(excel_3.index[[0, 1, 2, 3, 4, 5, 6]])

In [7]:
cont = 0
for a in excel_3.keys():
    for item in excel_3[a].isna():
        if item == True:
            cont+=1
cont

170

In [8]:
data_final = excel_3.dropna()
data_final = data_final.reset_index()

In [9]:
cont = 0
for item in data_final.keys():
    cont_2 = 0
    for a in data_final[item]:
        if cont>2:
            data_final[item][cont_2] = round(a)
            cont_2+=1
        
    cont+=1

<ipython-input-9-6a49630cf909>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final[item][cont_2] = round(a)


In [10]:
data_data = data_final.drop('nan', axis = 'columns')
data_data = data_data.drop('index', axis = 'columns')

In [13]:
ingresos_df = data_data[33::]
ingresos_df = ingresos_df.reset_index()
ingresos_df = ingresos_df.drop('index', axis = 'columns')
ingresos_df

,,2006,2009,2011,2013,2015,2017,2020
0,I,138991.0,145669.0,161288.0,195235.0,203882.0,207111.0,124479.0
1,II,266553.0,285498.0,300703.0,346835.0,372383.0,386743.0,306153.0
2,III,350790.0,360429.0,378165.0,440471.0,470933.0,484261.0,430462.0
3,IV,427467.0,452607.0,463986.0,518746.0,552571.0,572052.0,500747.0
4,V,497262.0,499406.0,529955.0,605889.0,659830.0,668418.0,600590.0
5,VI,591580.0,600425.0,607644.0,738608.0,765271.0,824982.0,741748.0
6,VII,737367.0,734865.0,737540.0,846903.0,902091.0,930052.0,877522.0
7,VIII,904200.0,895016.0,913235.0,1037176.0,1117492.0,1135564.0,1104125.0
8,IX,1251713.0,1252947.0,1257543.0,1477243.0,1468032.0,1556174.0,1611216.0
9,X,2806836.0,2898267.0,2856677.0,3258746.0,3268921.0,3507797.0,3433336.0


### Rendimiento estudiantil

In [16]:
dat=pd.read_csv('Propuesta/20210223_Rendimiento_2020_20210131_WEB.csv',delimiter=';')

C:\Users\anton\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
len(dat)

3164534

In [18]:
cols=dat.columns
for c in cols:
    print(c)

AGNO
RBD
DGV_RBD
NOM_RBD
COD_REG_RBD
NOM_REG_RBD_A
COD_PRO_RBD
COD_COM_RBD
NOM_COM_RBD
COD_DEPROV_RBD
NOM_DEPROV_RBD
COD_DEPE
COD_DEPE2
RURAL_RBD
ESTADO_ESTAB
COD_ENSE
COD_ENSE2
COD_GRADO
LET_CUR
COD_JOR
COD_TIP_CUR
COD_DES_CUR
MRUN
GEN_ALU
FEC_NAC_ALU
EDAD_ALU
COD_REG_ALU
COD_COM_ALU
NOM_COM_ALU
COD_RAMA
COD_SEC
COD_ESPE
PROM_GRAL
ASISTENCIA
SIT_FIN
SIT_FIN_R
COD_MEN


#### Eliminación y cambio de nombre de columnas

In [19]:
keep_cols = ['AGNO', 'RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_DEPE2',
            'RURAL_RBD', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO', 'PROM_GRAL', 'SIT_FIN_R']
dat = dat[keep_cols]
dat.reset_index(drop=True, inplace=True)

In [20]:
dat.tail()

,AGNO,RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_DEPE2,RURAL_RBD,COD_ENSE,COD_ENSE2,COD_GRADO,PROM_GRAL,SIT_FIN_R
3164529,2020,41807,INSTITUTO DE ENSEÑANZA PRIMARIA PROFESOR PAULO...,13,RM,3,0,110,2,1,"6,8",P
3164530,2020,41807,INSTITUTO DE ENSEÑANZA PRIMARIA PROFESOR PAULO...,13,RM,3,0,110,2,1,"6,9",P
3164531,2020,41807,INSTITUTO DE ENSEÑANZA PRIMARIA PROFESOR PAULO...,13,RM,3,0,110,2,2,7,P
3164532,2020,41807,INSTITUTO DE ENSEÑANZA PRIMARIA PROFESOR PAULO...,13,RM,3,0,110,2,2,"6,9",P
3164533,2020,41807,INSTITUTO DE ENSEÑANZA PRIMARIA PROFESOR PAULO...,13,RM,3,0,110,2,2,7,P


In [21]:
dat.rename(columns={'AGNO': 'AÑO', 'COD_DEPE2': 'COD_DEPENDENCIA', 'COD_ENSE': 'COD_ENSEÑANZA',
                   'COD_ENSE2': 'COD_ENSEÑANZA2', 'SIT_FIN_R': 'SIT_FINAL'}, inplace=True)
print(len(dat))
dat['AÑO'].unique()

3164534


array([2020], dtype=int64)

In [22]:
dat.drop('AÑO',axis = 1, inplace=True)

In [23]:
# Ciclo que recorre una lista con los códigos de grados de enseñanza básica
for cod in [110, 160, 161, 163, 165, 167]:
    # Se eliminan todos los registros con el código "cod" de la iteración
    dat=dat[dat['COD_ENSEÑANZA'] != cod]
dat.reset_index(drop=True, inplace=True)
len(dat)

1060244

In [24]:
# Ciclo que recorre una lista con los códigos de grados de enseñanza básica
for cod in [6, 8]:
    # Se eliminan todos los registros con el código "cod" de la iteración
    dat=dat[dat['COD_ENSEÑANZA2'] != cod]
dat.reset_index(drop=True, inplace=True)
len(dat)

933871

#### Datos faltantes

In [31]:
dat = dat.replace(' ', np.nan)
dat.isna().sum()

RBD                  0
NOM_RBD              0
COD_REG_RBD          0
NOM_REG_RBD_A        0
COD_DEPENDENCIA      0
RURAL_RBD            0
COD_ENSEÑANZA        0
COD_ENSEÑANZA2       0
COD_GRADO            0
PROM_GRAL            0
SIT_FINAL          141
dtype: int64

In [32]:
dat['SIT_FINAL'].unique()

array(['P', 'T', 'R', 'Y', nan], dtype=object)

In [33]:
data = pd.get_dummies(data=dat, columns=['SIT_FINAL'])
data.head()

,RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_DEPENDENCIA,RURAL_RBD,COD_ENSEÑANZA,COD_ENSEÑANZA2,COD_GRADO,PROM_GRAL,SIT_FINAL_P,SIT_FINAL_R,SIT_FINAL_T,SIT_FINAL_Y
0,1,LICEO POLITECNICO ARICA,15,AYP,5,0,510,7,1,"6,2",1,0,0,0
1,1,LICEO POLITECNICO ARICA,15,AYP,5,0,510,7,1,"6,8",1,0,0,0
2,1,LICEO POLITECNICO ARICA,15,AYP,5,0,510,7,1,"4,4",1,0,0,0
3,1,LICEO POLITECNICO ARICA,15,AYP,5,0,510,7,1,0,0,0,1,0
4,1,LICEO POLITECNICO ARICA,15,AYP,5,0,510,7,1,"5,9",1,0,0,0


In [34]:
keep_cols2 = ['RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_DEPENDENCIA',
            'RURAL_RBD', 'COD_ENSEÑANZA', 'COD_ENSEÑANZA2', 'COD_GRADO', 'SIT_FINAL_P']
data = data[keep_cols2]
data.reset_index(drop=True, inplace=True)

In [35]:
data['SIT_FINAL_P'].value_counts()

1    883117
0     50754
Name: SIT_FINAL_P, dtype: int64

In [36]:
data['COD_REG_RBD'].unique()

array([15,  1,  2,  3,  4,  5,  6,  7, 16,  8,  9, 14, 10, 11, 12, 13],
      dtype=int64)

In [55]:
a = pd.DataFrame(data.groupby('COD_REG_RBD')['SIT_FINAL_P'].sum())

In [56]:
b = pd.DataFrame(data['COD_REG_RBD'].value_counts().sort_index())

In [60]:
fin = b.join(a)

In [65]:
c = pd.DataFrame(fin['COD_REG_RBD']-fin['SIT_FINAL_P'])
final = fin.join(c)
final = final.drop('COD_REG_RBD', axis = 'columns')
final = final.rename(columns = {'SIT_FINAL_P':'APROBADOS', 0:'REPROBADOS'})
final

,APROBADOS,REPROBADOS
1,18673.0,910.0
2,32874.0,1610.0
3,15810.0,1048.0
4,40225.0,1444.0
5,87942.0,4808.0
6,47991.0,2902.0
7,55025.0,2927.0
8,80456.0,3896.0
9,52506.0,3489.0
10,44302.0,4068.0
